### **Q1. Running Elastic**

In [1]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

info = es_client.info()

print(f"Build hash: {info['version']['build_hash']}")

Build hash: 42f05b9372a9a4a470db3b52817899b99a76ee73


**Getting the data**

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### **Q2. Indexing the data**

In [3]:
from tqdm.auto import tqdm

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-week-01"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-week-01'})

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Question 3. Searching

In [7]:
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [13]:
top_score = response['hits']['hits'][0]['_score']
print(f'Top ranking result: {top_score}')

Top ranking result: 44.50556


### **Q4. Filtering**

In [19]:
import json

In [16]:
def retrieve_context(query, k, filters={}):
    search_query = {
        "size": k,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": filters
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [17]:
filters = {
    "course": "machine-learning-zoomcamp"
}

query = "How do copy a file to a Docker container?"

In [20]:
relevant_docs = retrieve_context(query, 3, filters)

print(json.dumps(relevant_docs, indent=4, ensure_ascii=False))

[
    {
        "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
        "section": "5. Deploying Machine Learning Models",
        "question": "How do I debug a docker container?",
        "course": "machine-learning-zoomcamp"
    },
    {
        "text": "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
        "section": "5. Deploying Machine Learning Models",
        "question": "How d

In [23]:
relevant_docs[-1].get('question')

'How do I copy files from a different folder into docker container’s working directory?'

### **Q5. Building a prompt**

In [40]:
context_template = "Q: {question}\nA: {text}"
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = "\n\n".join([
    context_template.format(question=r["question"], text=r["text"]) for r in relevant_docs
])

prompt = prompt_template.format(question=query, context=context)

print(f'Length of the resulting prompt {len(prompt)}')

Length of the resulting prompt 1446


In [41]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

### Q6. Tokens

In [42]:
import tiktoken

In [43]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [44]:
tokens = encoding.encode(prompt)

print("Tokens Number:", len(tokens))

Tokens Number: 320


In [53]:
token = tokens[0]

decoded_bytes = encoding.decode_single_token_bytes(token)
print("Bytes:", decoded_bytes)
print("String:", decoded_bytes.decode("utf-8"))

Bytes: b"You're"
String: You're


### **Bonus: generating the answer**

In [57]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [58]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [64]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [65]:
llm_response = llm(prompt)

print(llm_response)

To copy a file from your local machine to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


In [66]:
output_tokens = encoding.encode(llm_response)

print("Tokens Number:", len(output_tokens))

Tokens Number: 49


### **Bonus: calculating the costs**

For 1000 requests:

- Total input tokens = 150 tokens/request * 1000 requests = 150,000 tokens.
- Total output tokens = 250 tokens/request * 1000 requests = 250.000 tokens

Input Cost:

- 150,000 tokens * ($0.005 / 1000) = 0.75 USD

Output Cost:

- 250,000 tokens * ($0.015 / 1000) = 3.75 USD

Total Output:
- 0.75 (input) + 3.75 (output) = **4.50 USD**

For execution:

Input Cost:

- 320 tokens * ($0.005 / 1000) = 0.0016 USD

Output Cost:

- 49 tokens * ($0.015 / 1000) = 0.000735 USD

Total Output:
- 0.0016 (input) + 0.000735 (output) = **0.002335 USD**